Criação dos dados

In [18]:
import numpy as np
import pandas as pd


np.random.seed(0)
n_samples = 300

data = {
    'valor_medio_gasto': np.random.randn(n_samples)*20 +200,
    'frequencia_compras': np.random.randn(n_samples)*5 +15,
    'categorias_produtos': np.random.randn(n_samples)*2 +8

}

df = pd.DataFrame(data)
print(df)

     valor_medio_gasto  frequencia_compras  categorias_produtos
0           235.281047            8.467366             4.899141
1           208.003144           23.290653             8.834638
2           219.574760           14.409180             6.111263
3           244.817864           11.599109             8.476206
4           237.351160           18.331915             5.188074
..                 ...                 ...                  ...
295         222.737827           16.896178            11.995912
296         201.954499           12.649836             6.286901
297         211.659074           13.916343             4.916825
298         192.011019           10.349217            13.188849
299         207.401118           14.107055             7.191935

[300 rows x 3 columns]


Implementando Kmeans manualmente

In [19]:
def inicializar_centroide(X,k):
  num_amostras, num_atrib = X.shape
  centroides = []

  prim_centroide = np.random.randint(0,num_amostras)
  centroides.append(X.iloc[prim_centroide]) # Use .iloc for DataFrame indexing

  for _ in range(1,k):
    distancia_min = np.array([min(np.sum((x-c)**2) for c in centroides) for x in X.values]) # Use .values for NumPy array operations
    probabilidade = distancia_min / np.sum(distancia_min)
    probabilidade_acumulada = np.cumsum(probabilidade)
    r = np.random.rand()

    for i,p in enumerate(probabilidade_acumulada):
      if r < p:
        centroides.append(X.iloc[i])
        break
  return np.array(centroides)

def clusters_assinatura(X,centroides):
  clusters = []
  for x in X.values:
    distancias = np.linalg.norm(x-centroides,axis=1)
    clusters.append(np.argmin(distancias))
  return np.array(clusters)


def atualizar_centroides(X,clusters,k):
  novos_centroides = []
  for i in range(k):
    points = X.values[clusters == i ]
    if len(points)>0:
      novos_centroides.append(points.mean(axis=0))
    else:
      novos_centroides.append(np.zeros(X.shape[1]))
  return np.array(novos_centroides)


def kmeans_pp(X,k,max_iter=100,tol=1e-4):
  centroides = inicializar_centroide(X,k)

  for _ in range(max_iter): # Corrected variable name
    clusters = clusters_assinatura(X,centroides)
    novos_centroides = atualizar_centroides(X,clusters,k)

    if np.allclose(centroides,novos_centroides,atol=tol):
      break
    centroides = novos_centroides
  return centroides,clusters

Resultados

In [21]:
k = 3


centroides, clusters = kmeans_pp(df, k)

print("Centróides finais:")
print(centroides)
print("Cluster de cada ponto:")
print(clusters)

Centróides finais:
[[200.96227388  14.96377275   7.9000788 ]
 [177.21676415  13.67924579   7.80765033]
 [225.84961747  13.92276763   7.65654028]]
Cluster de cada ponto:
[2 0 2 2 2 1 2 0 0 0 0 2 2 0 0 0 2 0 0 1 1 0 2 1 2 1 0 0 2 2 0 0 1 1 0 0 2
 2 0 0 1 1 1 2 0 0 1 2 1 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 1 0 2 0 2 1
 0 1 1 1 0 0 1 2 0 1 2 2 2 0 1 2 0 2 0 2 0 2 0 2 0 0 2 1 1 2 1 2 0 1 2 2 2
 2 1 2 0 2 2 0 0 2 0 1 0 2 1 0 0 2 2 0 1 0 1 0 1 2 0 0 0 1 1 0 0 0 2 2 1 2
 1 0 0 2 1 1 0 1 2 1 1 0 0 2 2 0 1 2 1 1 2 0 2 0 2 1 1 2 1 1 0 0 0 1 1 1 0
 1 1 0 1 2 1 0 0 1 0 0 2 2 2 2 0 0 2 0 0 1 0 1 0 0 2 0 2 0 1 0 0 0 2 0 1 0
 0 0 1 0 0 0 1 0 1 0 0 0 1 2 2 1 0 2 1 0 0 0 0 1 2 2 1 1 0 1 0 0 2 2 1 1 1
 0 1 0 1 0 1 0 0 0 0 0 0 1 2 0 1 0 0 0 1 2 0 2 1 2 0 0 1 2 2 2 1 0 2 1 0 2
 0 0 0 0]


In [23]:
print(sum(clusters == 0))
print(sum(clusters == 1))
print(sum(clusters == 2))

133
88
79


Optei por utilizar o Kmeans por ser um conjunto de dados pequenos, sem muito outliers e termos somente 3 variaveis para analisar e montar nosso modelo. Visto a praticidade e o fato de não termos um volume tão grande de dados para analisar optei por utilizar o modelo em questão.
Esse modelo possui como principal vantagem sem de fácil entedimento, fácil de aplicar e rápido de implementar. Uma desvantagem dele é que se o numero de variaveis analisadas aumentar para 5,6,7, etc. teriamos um custo computacional muito grande, pois creio que seriam necessarias muito mais iterações para acharmos o cluster ideal.
Os cluster em questão podem ser usados para fazer a segmentação dos dados dos clientes para otimizar promoções, agrupar grupo de produtos que são vendidos juntos com mais frenquência entre outros.